<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/autosklearn/freedom/04A_Autosklearn_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install gcc
!gcc --version
!apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
!pip install pmlb
!pip install pytictoc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/unive

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = pd.DataFrame(truth)
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

class CheckNegativeValue:
  def __init__(self, checknegative):
    self._is_negative = "neg" if np.min(checknegative) < 1 else "pos"
  
  def MLMetrics(self):
    if self._is_negative == "neg" :
      return [metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2]
    elif self._is_negative == "pos" :
      return ['neg_mean_squared_log_error', metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2]

In [3]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from autosklearn.estimators import AutoSklearnRegressor
import autosklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd
from pprint import pprint

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])

preds = {}
cm_preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded  in [metrics.mean_absolute_error]:
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        df_result.loc[ix,"dataset_name"] = rgrs_dtst 
        df_result.loc[ix,"metrics_recorded"] = metrics_recorded 
        X, y = fetch_data(rgrs_dtst , return_X_y=True)
        c=CheckNegativeValue(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        y_train = pd.DataFrame(y_train).values.ravel()
        y_test = pd.DataFrame(y_test).values.ravel()
        list_metrics = c.MLMetrics()
        automl = AutoSklearnRegressor(seed=69, n_jobs=4, ml_memory_limit=2048,time_left_for_this_task=1800)
        t.tic()
        automl.fit(X_train,y_train, metric=metrics_recorded if metrics_recorded in list_metrics else None)
        t.toc()
        inse = automl.show_models()
        try:
          res_json=eval(inse.split("(")[2].split("))")[0].split(",\ndataset_properties")[0])
        except:
          print('dummy_regressor')
        top_algo[rgrs_dtst] = res_json['regressor:__choice__']
        print("*** Top algorithm: ***", res_json['regressor:__choice__'])
        pprint(res_json)
        preds[rgrs_dtst] = automl.predict(X_test)
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
[WARNING] [2020-05-28 07:11:39,534:AutoMLSMBO(1272380470)::76c127f2259c67c8487eca3f68a8aaa5] Could not find meta-data directory /usr/local/lib/python3.6/dist-packages/autosklearn/metalearning/files/mean_absolute_error_regression_dense
Elapsed time is 1804.964257 seconds.
*** Top algorithm: *** k_nearest_neighbors
{'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding',
 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'no_coalescense',
 'data_preprocessing:numerical_transformer:imputation:strategy': 'median',
 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler',
 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_max': 0.7289204838378651,
 'data_preprocessing:numerical_transformer:rescaling:robust_scaler:q_min': 0.22303310656600822,
 'feature_preproc

In [4]:
for i, val in enumerate(preds):
  print(preds[val])

[7. 1. 1. 7. 5. 3. 1. 1. 4. 3. 7. 1. 4. 2. 2. 5. 2. 4. 1. 3. 3. 7. 1. 3.
 1. 3. 7. 1. 7. 4. 3. 1. 2. 4. 5. 3. 7. 1. 5. 1. 5. 3. 1. 7. 1. 1. 3. 7.
 1. 4. 5. 1. 5. 3. 3. 4. 7. 3. 5. 4. 3. 3. 1. 7. 1. 7. 2. 3. 2. 3. 1. 2.
 3. 5. 7. 1. 2. 5. 7. 7. 3. 7. 1. 3. 7. 7. 5. 7. 3. 1. 4. 3. 3. 3. 3. 1.
 7. 3. 1. 3. 7. 7. 1. 3. 1. 7. 1. 7. 3. 2. 2. 4. 7. 1. 4. 7. 2. 1. 1. 3.
 7. 1. 5. 5. 1. 3. 7. 2. 7. 1. 1. 1. 1. 3. 2. 2. 4. 7. 3. 4. 1. 1. 2. 2.
 5. 7. 3. 4. 3. 5. 7. 1. 7. 7. 7. 4. 4. 7. 4. 7. 3. 7. 1. 5. 1. 5. 5. 2.
 1. 3. 7. 5. 1. 3. 2. 1. 3. 3. 7. 4. 3. 7. 3. 4. 5. 1. 1. 1. 7. 2. 5. 7.
 7. 1. 1. 3. 2. 1. 3. 3. 7. 7. 4. 1. 3. 1. 3. 4. 1. 3. 5. 3. 1. 5. 5. 5.
 7. 5. 3. 7. 5. 5. 3. 2. 4. 5. 7. 3. 2. 2. 1. 7. 3. 3. 4. 4. 3. 7. 2. 1.
 7. 4. 3. 5. 7. 3. 2. 5. 1. 3. 1. 5. 1. 2. 3. 3. 1. 5. 3. 1. 1. 1. 1. 2.
 4. 7. 1. 1. 1. 3. 1. 7. 7. 7. 1. 4. 1. 1. 7. 1. 7. 3. 5. 1. 7. 2. 7. 3.
 7. 7. 7. 3. 7. 3. 3. 1. 3. 3. 7. 7. 1. 4. 3. 7. 2. 4. 1. 1. 1. 7. 2. 7.
 1. 1. 3. 7. 5. 7. 5. 7. 7. 1. 5. 5. 5. 3. 1. 1. 7.

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
pprint(df_result)

          dataset_name     metrics_recorded       MAE          MSE       MSLE  \
0  294_satellite_image  mean_absolute_error  0.270186  0.776398     0.0316149   
1  218_house_8L         mean_absolute_error  15505.1   8.51371e+08  0.390528    
2  227_cpu_small        mean_absolute_error  2.10014   9.30338      NaN         
3  503_wind             mean_absolute_error  2.2627    8.75814      0.0500027   
4  344_mv               mean_absolute_error  0.851337  1.16968      NaN         
5  215_2dplanes         mean_absolute_error  0.795138  1.00908      NaN         

       RMSE     RMSLE        r2  
0  0.881134  0.177806  0.839525  
1  29178.3   0.624923  0.671057  
2  3.05014   NaN       0.970674  
3  2.95941   0.223613  0.801999  
4  1.08152   NaN       0.989073  
5  1.00453   NaN       0.94718   


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [0]:
from google.colab import files

with open('data_5.csv', 'w') as f:
    f.write(str(df_result))

with open('data_5.txt', 'w') as f:
    f.write(str(top_algo))

#files.download('data_5.txt')